In [149]:
import pandas as pd
import scipy 
import numpy as np
import os
data=pd.read_csv('Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
print data.shape
print data[' Label'].unique()
data=data.drop(['Flow Bytes/s',' Flow Packets/s'],axis=1)

(225745, 79)
['BENIGN' 'DDoS']


In [152]:
print data.shape
for i in data.columns:
    tmp=np.asarray(data[i])
    if(np.count_nonzero(tmp)==0):
        print("Deleting ",i)
        data=data.drop(i,axis=1)
    


(225745, 77)
('Deleting ', ' Bwd PSH Flags')
('Deleting ', ' Fwd URG Flags')
('Deleting ', ' Bwd URG Flags')
('Deleting ', ' CWE Flag Count')
('Deleting ', 'Fwd Avg Bytes/Bulk')
('Deleting ', ' Fwd Avg Packets/Bulk')
('Deleting ', ' Fwd Avg Bulk Rate')
('Deleting ', ' Bwd Avg Bytes/Bulk')
('Deleting ', ' Bwd Avg Packets/Bulk')
('Deleting ', 'Bwd Avg Bulk Rate')


In [155]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
data_=np.asarray(data)
length=len(data_[0])
X=data_[:,0:length-1]
Labels=data_[:,-1]
le = preprocessing.LabelEncoder()
Y=le.fit_transform(Labels)
#print Y
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)
print np.shape(X)
print np.shape(Y)

(225745L, 66L)
(225745L,)


In [154]:
cols_to_norm = data.columns
cols_to_norm=cols_to_norm[1:len(cols_to_norm)-1]
print len(cols_to_norm)
print cols_to_norm
data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

65
Index([u' Flow Duration', u' Total Fwd Packets', u' Total Backward Packets',
       u'Total Length of Fwd Packets', u' Total Length of Bwd Packets',
       u' Fwd Packet Length Max', u' Fwd Packet Length Min',
       u' Fwd Packet Length Mean', u' Fwd Packet Length Std',
       u'Bwd Packet Length Max', u' Bwd Packet Length Min',
       u' Bwd Packet Length Mean', u' Bwd Packet Length Std',
       u' Flow IAT Mean', u' Flow IAT Std', u' Flow IAT Max', u' Flow IAT Min',
       u'Fwd IAT Total', u' Fwd IAT Mean', u' Fwd IAT Std', u' Fwd IAT Max',
       u' Fwd IAT Min', u'Bwd IAT Total', u' Bwd IAT Mean', u' Bwd IAT Std',
       u' Bwd IAT Max', u' Bwd IAT Min', u'Fwd PSH Flags',
       u' Fwd Header Length', u' Bwd Header Length', u'Fwd Packets/s',
       u' Bwd Packets/s', u' Min Packet Length', u' Max Packet Length',
       u' Packet Length Mean', u' Packet Length Std',
       u' Packet Length Variance', u'FIN Flag Count', u' SYN Flag Count',
       u' RST Flag Count', u' PSH Flag 

In [156]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)

#Predict Output
predicted= model.predict(X_test)

In [157]:
from sklearn.metrics import classification_report
print predicted
print y_test
print classification_report(y_test, predicted)

[1 0 1 ... 1 1 1]
[1 0 0 ... 1 1 1]
             precision    recall  f1-score   support

          0       1.00      0.42      0.59     29262
          1       0.69      1.00      0.82     38462

avg / total       0.83      0.75      0.72     67724



In [158]:
print np.shape(y_test)
print np.shape(predicted)

(67724L,)
(67724L,)


In [159]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predicted)
#print data['Flow Bytes/s']

0.7502214872128049

[0 0 0 ... 0 0 0]


(225745L, 77L)
(225745L,)


In [112]:
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [128]:
for i in data.columns:
    if(data[i].isnull().values.any()):
        print i

In [72]:
col_issue=data['Flow Bytes/s']
print type(col_issue)
col_issue=np.asarray(col_issue)

<class 'pandas.core.series.Series'>


In [73]:
normalized_df=(col_issue-col_issue.min())/(col_issue.max()-col_issue.min())

C:\Program Files (x86)\Anaconda2\lib\site-packages\numpy\core\_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)


TypeError: unsupported operand type(s) for -: 'str' and 'float'

TypeError: ("unsupported operand type(s) for -: 'str' and 'float'", u'occurred at index  Flow Packets/s')

In [119]:
print data['Flow Packets/s']

KeyError: 'Flow Packets/s'